#  Trademark Budget Automation Tool

Welcome to the **Trademark Budget Automation Tool**, a streamlined Python-based workflow designed to simplify the generation of **budget estimates** and **procedural documentation** for **trademark applications** and **renewals**. This tool delivers high-quality outputs in both **PDF** and **DOCX** formats, ensuring a clean and professional user experience.

---

## 📌 Key Features

- 💼 Supports **both single and multiple trademark** scenarios
- 🌍 Handles **international filings** by dynamically adjusting for country and correspondent
- 🧾 Generates:
  - Budget Summary PDFs
  - Procedural Detail PDFs
  - Formal DOCX documents based on deposit type and classification
- 🚀 Simple, guided user interaction
- 🔐 Built-in failsafe checks, subprocess logging, and file naming sanitization

---

## 🤖 Purpose

This notebook is intended for IP professionals, consultants, and internal legal departments who require **quick, repeatable, and scalable** document generation for trademark filings.

---

## 👤 Author

Developed and maintained by:

**Martin G. Lartigue**  
📧 martin.g.lartigue@gmail.com

Feel free to reach out for inquiries, suggestions, or collaboration opportunities.

---

> ⚠️ *Note:* This tool assumes basic familiarity with trademark terminology and filing structures. For best results, ensure LaTeX and the required templates are properly configured in your environment.


## ⚙️ Environment Setup & Function Definitions

This section initializes the runtime environment and defines all core functions required for the workflow.

### 🛠️ Setup
We begin by importing the necessary libraries and configuring key settings (e.g., display options, runtime parameters, or file paths) to ensure the notebook runs as expected.

### 🔁 Function Definitions
Custom helper functions are declared here to modularize the codebase and improve reusability, readability, and debugging. These functions typically support data preprocessing, transformation, or analysis tasks.

> 📌 *Note:* Ensure this cell is executed before running any downstream logic, as subsequent cells depend on these configurations and definitions.


### ⚙️ Setup & Initialization – Budget Tool for Trademarks

This code block prepares the Colab environment for generating trademark application and renewal budgets. It:

- Imports essential libraries for data handling, formatting, file access, and document generation.
- Installs required packages (`python-docx` and LaTeX with Portuguese support).
- Mounts Google Drive to access templates and save outputs.
- Defines file paths and constants for Excel data sources, Word templates, and output directories.

This setup ensures the system is ready to automate the creation of branded, localized budget documents.


In [56]:
# Import necessary libraries for various functionalities:
# - pandas: For data manipulation and handling Excel spreadsheets
# - unicodedata: For text normalization (e.g., removing accents)
# - tabulate: For pretty-printing tables in the console
# - gdown: For downloading files from Google Drive (not used here but imported)
# - os: For interacting with the operating system (e.g., file paths)
# - google.colab.drive: For mounting Google Drive in Colab environment
# - datetime and pytz: For handling dates and timezones (Brasília time)
# - re: For regular expression operations (e.g., sanitizing filenames)
# - subprocess: For running shell commands (e.g., compiling LaTeX to PDF)
# - math: For mathematical operations (e.g., ceiling for table formatting)
import pandas as pd
import unicodedata
from tabulate import tabulate
import gdown
import os
from google.colab import drive
from datetime import datetime
import pytz
import re
import subprocess
import math

# Install required external libraries:
# - python-docx: For generating and manipulating DOCX files
!pip install python-docx
# - LaTeX packages: For generating PDF documents with XeLaTeX, including Portuguese language support
!apt-get update
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-latex-recommended texlive-lang-portuguese

# Import Document class from python-docx after installation to work with DOCX files
from docx import Document

# Mount Google Drive to access spreadsheet and template files stored there
drive.mount('/content/drive')

# Set up the output directory for saving generated budgets:
# - Creates the directory if it doesn't exist to avoid file writing errors
output_dir = "/content/drive/MyDrive/INSERT_DESIRED_OUTPUT_PATH"
os.makedirs(output_dir, exist_ok=True)

# Define constants used throughout the script:
# - EXIT_KEYWORD: Special keyword to exit the program at any input prompt
# - FILE_PATH: Path to the Excel spreadsheet containing fee and correspondent data
# - DOCX_TEMPLATE_PATH: Path to the DOCX template for single-class trademark deposits
# - DOCX_MULTI_CLASS_TEMPLATE_PATH: Path to the DOCX template for multi-class trademark deposits
# - DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH: Path to the DOCX template for single-class ordinary extensions
# - DOCX_PRORROGACAO_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH: Path to the DOCX template for multi-class ordinary extensions
# - DOCX_PRORROGACAO_EXTRA_ORDINARIO_TEMPLATE_PATH: Path to the DOCX template for single-class extraordinary extensions
# - DOCX_PRORROGACAO_EXTRA_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH: Path to the DOCX template for multi-class extraordinary extensions
# - LOGO_PATH: Path to the logo image included in generated PDFs
EXIT_KEYWORD = "SAIR"
FILE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/SCHEDULE OF FEES.xlsx"
DOCX_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Deposito_1_Classe.docx"
DOCX_MULTI_CLASS_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Deposito_Multi_Classe.docx"
DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Prorrogacao_Ordinario_1_Classe.docx"
DOCX_PRORROGACAO_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Prorrogacao_Ordinario_Multi_Classe.docx"
DOCX_PRORROGACAO_EXTRA_ORDINARIO_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Prorrogacao_Extra_Ordinario_1_Classe.docx"
DOCX_PRORROGACAO_EXTRA_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/PLANILHAS/Word_Template_Prorrogacao_Extra_Ordinario_Multi_Classe.docx"
LOGO_PATH = "/content/drive/MyDrive/CODE/Projeto Schedule of Fees/Logo/logo.png"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

### 📊 Data Ingestion & Column Normalization – Budget Tool for Trademarks

This segment ensures clean and standardized column headers from the source Excel spreadsheet. It:

- Defines a normalization function that removes diacritics and replaces spaces with underscores for uniformity.
- Loads the spreadsheet into a DataFrame using the configured file path.
- Strips leading/trailing whitespace from column names.
- Applies the normalization function to all column headers to ensure consistency and compatibility with downstream processing.
- Outputs the transformed column names to the console for debugging and validation purposes.

This step mitigates risks related to inconsistent header formatting and improves reliability in data mapping and transformation.


In [57]:
# Define a dictionary mapping month numbers to Portuguese month names for date formatting
month_names = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

# Initialize an empty list to track DOCX files generated during this execution
generated_docx_files = []

# Function to normalize column names by removing accents and replacing spaces with underscores
def normalize_column(col):
    """
    Normalize a column name for consistent use in code.

    Parameters:
        col (str): The original column name from the spreadsheet.

    Returns:
        str: Normalized column name with accents removed and spaces replaced by underscores.
    """
    nfkd = unicodedata.normalize('NFKD', col)
    without_accent = "".join([c for c in nfkd if not unicodedata.combining(c)])
    return without_accent.replace(" ", "_")

# Load the Excel spreadsheet containing fee data and normalize its column names
sheet_df = pd.read_excel(FILE_PATH)
sheet_df.columns = sheet_df.columns.str.strip()  # Remove leading/trailing whitespace from column names
sheet_df.columns = [normalize_column(col) for col in sheet_df.columns]  # Apply normalization to all columns
# Debug: Display normalized column names to verify correct loading and formatting
print("Debug: Excel column names after normalization:", list(sheet_df.columns))

Debug: Excel column names after normalization: ['Ultima_Atualizacao', 'Nome_do_Correspondente', 'Pais_de_Atuacao', 'Peculiaridades_de_Tramites', 'Descricao_dos_Documentos_Necessarios', 'Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'Busca', 'Deposito_de_Marca_Mista_1_Classe', 'Deposito_de_Marca_Mista_Classe_Adicional', 'Deposito_de_Marca_Nominativa_1_Classe', 'Deposito_de_Marca_Nominativa_Classe_Adicional', 'Deposito_de_Marca_Figurativa_1_Classe', 'Deposito_de_Marca_Figurativa_Classe_Adicional', 'Concessao_de_Marca_1_Classe', 'Concessao_de_Marca_Classe_Adicional', 'Prorrogacao_Prazo_Ordinario_1_Classe', 'Prorrogacao_Prazo_Ordinario_Classe_Adicional', 'Prorrogacao_Prazo_Extra_Ordinario_1_Classe', 'Prorrogacao_Prazo_Extra_Ordinario_Classe_Adicional']


### 📅 Date Localization & Formatting – Budget Tool for Trademarks

This block localizes and formats the current date to align with Brazilian conventions. It:

- Maps Gregorian month numbers to their corresponding names in Portuguese.
- Retrieves the current date and time based on the Brasília timezone (`America/Sao_Paulo`).
- Extracts the day, localized month name, and year to construct a human-readable date string (e.g., `16 de Junho de 2025`).
- Generates a compact, filename-safe version of the date in `DD-MM-YYYY` format for use in output file naming.

This ensures generated documents reflect accurate, locale-appropriate timestamps and maintain consistent naming conventions.


In [58]:
# Portuguese month names
month_names = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

# Get the current date in Brasília timezone for use in documents
brasilia_tz = pytz.timezone('America/Sao_Paulo')
current_date = datetime.now(brasilia_tz)
day = f"{current_date.day:02d}"  # Ensure day is two digits (e.g., 01)
month = month_names[current_date.month]  # Convert month number to Portuguese name
year = str(current_date.year)
formatted_date = f"{day} de {month} de {year}"  # Format date as "DD de Month de YYYY"
# Format date as DD-MM-YYYY for use in filenames
filename_date = current_date.strftime("%d-%m-%Y")


### 🧾 Fee Structure & Document Templates – Budget Tool for Trademarks

This component establishes the baseline pricing and LaTeX layout logic for generating branded trademark budget reports. It:

- Defines `honorarios_dict`, a centralized fee reference of the user's company for various trademark-related services, including application, renewal, and opposition actions.
- Declares reusable LaTeX templates for three document categories:
  - **Depósito** (application) budget pages.
  - **Prorrogação** (renewal) budget pages.
  - **Details** pages with correspondent-specific procedural and documentation information. To allow the user to validate if the particular budget should be done by hand instead of automate before sending to the client (ex: Outdated values in correspondent fees)
- Each template ensures:
  - Portuguese-language support via the `babel` package.
  - Standardized header and footer styling with logos and corporate contact details.
  - Consistent color scheme and formatting for tables and section headers using custom LaTeX column types and the `readerred` brand color.
  - Structured placeholders (`%s`) for dynamic insertion of client-specific and case-specific data.

This setup guarantees professional, localized, and brand-aligned documentation output across multiple budget types and use cases.


In [59]:
# Define a dictionary of professional fees (honorários) for various services in USD Fictional Values
honorarios_dict = {
    "Honorarios_Busca": 120,
    "Honorarios_Deposito_de_Marca_Mista_1_Classe": 410,
    "Honorarios_Deposito_de_Marca_Mista_Classe_Adicional": 200,
    "Honorarios_Deposito_de_Marca_Nominativa_1_Classe": 380,
    "Honorarios_Deposito_de_Marca_Nominativa_Classe_Adicional": 200,
    "Honorarios_Deposito_de_Marca_Figurativa_1_Classe": 380,
    "Honorarios_Deposito_de_Marca_Figurativa_Classe_Adicional": 200,
    "Honorarios_Oposicao_a_Deposito_de_Marca": 350,
    "Honorarios_Concessao_de_Marca_1_Classe": 350,
    "Honorarios_Concessao_de_Marca_Classe_Adicional": 300,
    "Honorarios_Prorrogacao_Prazo_Ordinario_1_Classe": 500,
    "Honorarios_Prorrogacao_Prazo_Extra_Ordinario_1_Classe": 700,
    "Honorarios_Prorrogacao_Prazo_Ordinario_Classe_Adicional": 300,
    "Honorarios_Prorrogacao_Prazo_Extra_Ordinario_Classe_Adicional": 600,
}

# Define the LaTeX template for the budget document (used for PDF generation)
latex_template_deposito = """
%% Main document template for budget PDFs
\\documentclass[a4paper,12pt]{article}  %% Set document class to article with A4 paper and 12pt font
\\usepackage[utf8]{inputenc}  %% Use UTF-8 encoding for input text
\\usepackage[T1]{fontenc}  %% Use T1 font encoding for better character support
\\usepackage{geometry}  %% Customize page layout
\\geometry{margin=2.2cm, headsep=1.5cm}  %% Set margins and header separation
\\usepackage{booktabs}  %% Enhance table formatting
\\usepackage{array}  %% Improve table column definitions
\\usepackage{colortbl}  %% Add color to tables
\\usepackage{xcolor}  %% Define and use custom colors
\\usepackage{parskip}  %% Adjust paragraph spacing
\\usepackage{lastpage}  %% Reference the last page number
\\usepackage{fancyhdr}  %% Customize headers and footers
\\usepackage[brazil]{babel}  %% Set language to Brazilian Portuguese
\\usepackage{graphicx}  %% Include images (e.g., logo)
%% Configure fonts to use Times New Roman
\\usepackage{times}
%% Define custom color for branding (Company custom color)
\\definecolor{readerred}{RGB}{207,45,75}
%% Set up header and footer layout
\\setlength{\\headheight}{70pt}  %% Increase header height to fit content
\\addtolength{\\topmargin}{-10pt}  %% Adjust top margin
\\pagestyle{fancy}  %% Use fancy page style
\\fancyhf{}  %% Clear default header/footer content
\\fancyhead[L]{\\color{readerred}\\small\\textbf{Insert Company Name} \\\\[-0.1cm] CNPJ: 12.345.567/0000-01 \\\\[-0.1cm] Insert Company Address Zip Code: 000-00 \\\\[-0.1cm] Company Phone Contact | Company Email Contact}  %% Left header with company info
\\fancyhead[R]{\\raisebox{-0.5cm}{\\includegraphics[width=0.3\\textwidth]{logo.png}}}  %% Right header with logo
\\fancyfoot[C]{\\thepage\\ de \\pageref{LastPage}}  %% Center footer with page number
%% Define custom column types for tables
\\newcolumntype{R}[1]{>{\\raggedleft\\arraybackslash}p{#1}}  %% Right-aligned column
\\newcolumntype{L}[1]{>{\\raggedright\\arraybackslash}p{#1}}  %% Left-aligned column
%% Start the document content
\\begin{document}
%s  %% Placeholder for budget pages
\\end{document}
"""

# Define the LaTeX template for a single budget page for trademark deposits
latex_template_deposito_page = """
%% Budget page for a single trademark deposit
\\vspace*{-0.25cm}  %% Adjust vertical spacing at the top
\\begin{center}
    {\\LARGE \\textbf{Orçamento em %s}}\\\\  %% Title with country name
    \\vspace{0.5cm}
    {\\large Data: %s}  %% Date of the budget
\\end{center}
\\section*{Informações do Cliente}
\\begin{tabular}{ll}
    \\textbf{Titular:} & %s \\\\  %% Company name
    \\textbf{Marca:} & %s \\\\  %% Trademark name
\\end{tabular}
\\section*{Detalhamento do Orçamento}
\\begin{tabular}{|L{10cm}|R{3cm}|R{3cm}|}
    \\hline
    \\rowcolor{readerred}
    \\color{white}\\textbf{Descrição} & \\color{white}\\textbf{Taxa do Correspondente (US\\$)} & \\color{white}\\textbf{Valor Total (US\\$)} \\\\  %% Table header
    \\hline
%s  %% Table rows with cost breakdown
    \\multicolumn{2}{|r|}{\\textbf{Total Estimado Até Registro:}} & \\textbf{%s} \\\\  %% Total estimated cost
    \\hline
\\end{tabular}
\\section*{Observações}
Os valores acima informados referem-se às etapas principais do depósito de marca, por marca e por classe:
\\begin{itemize}
    \\item Primeira etapa: Busca,
    \\item Segunda etapa: Depósito,
    \\item Terceira etapa (quando ocorrer): Registro, expedição do Certificado de Registro, Acompanhamento e Vigilância por 10 anos.
\\end{itemize}
Disclaimer Text Explaining Trademark Application Process. For example one my point out that international budgets are volatile according to the stability of the currencies of the countries involved.
\\newpage
"""

# Define the LaTeX template for a single budget page for trademark extensions
latex_template_prorrogacao_page = """
%% Budget page for a single trademark extension
\\vspace*{-0.25cm}  %% Adjust vertical spacing at the top
\\begin{center}
    {\\LARGE \\textbf{Orçamento em %s}}\\\\  %% Title with country name
    \\vspace{0.5cm}
    {\\large Data: %s}  %% Date of the budget
\\end{center}
\\section*{Informações do Cliente}
\\begin{tabular}{ll}
    \\textbf{Titular:} & %s \\\\  %% Company name
    \\textbf{Marca:} & %s \\\\  %% Trademark name
\\end{tabular}
\\section*{Detalhamento do Orçamento}
\\begin{tabular}{|L{10cm}|R{3cm}|R{3cm}|}
    \\hline
    \\rowcolor{readerred}
    \\color{white}\\textbf{Descrição} & \\color{white}\\textbf{Taxa do Correspondente (US\\$)} & \\color{white}\\textbf{Valor Total (US\\$)} \\\\  %% Table header
    \\hline
%s  %% Table rows with cost breakdown
    \\multicolumn{2}{|r|}{\\textbf{Total:}} & \\textbf{%s} \\\\  %% Total cost
    \\hline
\\end{tabular}
\\section*{Observações}
OBSERVACAO SERA ESCRITA EM BREVE
\\newpage
"""

# Define the LaTeX template for the details document (used for PDF generation)
latex_template_details = """
%% Main document template for details PDFs
\\documentclass[a4paper,12pt]{article}  %% Set document class to article with A4 paper and 12pt font
\\usepackage[utf8]{inputenc}  %% Use UTF-8 encoding for input text
\\usepackage[T1]{fontenc}  %% Use T1 font encoding for better character support
\\usepackage{geometry}  %% Customize page layout
\\geometry{margin=2.2cm, headsep=1.5cm}  %% Set margins and header separation
\\usepackage{booktabs}  %% Enhance table formatting
\\usepackage{array}  %% Improve table column definitions
\\usepackage{colortbl}  %% Add color to tables
\\usepackage{xcolor}  %% Define and use custom colors
\\usepackage{parskip}  %% Adjust paragraph spacing
\\usepackage{lastpage}  %% Reference the last page number
\\usepackage{fancyhdr}  %% Customize headers and footers
\\usepackage[brazil]{babel}  %% Set language to Brazilian Portuguese
\\usepackage{graphicx}  %% Include images (e.g., logo)
%% Configure fonts to use Times New Roman
\\usepackage{times}
%% Define custom color for branding (Company Custom)
\\definecolor{readerred}{RGB}{207,45,75}
%% Set up header and footer layout
\\setlength{\\headheight}{70pt}  %% Increase header height to fit content
\\addtolength{\\topmargin}{-10pt}  %% Adjust top margin
\\pagestyle{fancy}  %% Use fancy page style
\\fancyhf{}  %% Clear default header/footer content
\\fancyhead[L]{\\color{readerred}\\small\\textbf{Insert Company Name} \\\\[-0.1cm] CNPJ: 12.345.567/0000-01 \\\\[-0.1cm] Insert Company Address Zip Code: 000-00 \\\\[-0.1cm] Company Phone Contact | Company Email Contact}  %% Left header with company info
\\fancyhead[R]{\\raisebox{0.0cm}{\\includegraphics[width=0.3\\textwidth]{logo.png}}}  %% Right header with logo
\\fancyfoot[C]{\\thepage\\ de \\pageref{LastPage}}  %% Center footer with page number
%% Define custom column type for tables
\\newcolumntype{L}[1]{>{\\raggedright\\arraybackslash}p{#1}}  %% Left-aligned column
\\begin{document}
%s  %% Placeholder for details pages
\\end{document}
"""

# Define the LaTeX template for a single details page about the correspondent
latex_template_details_page = """
%% Details page for a single correspondent
\\vspace*{-0.25cm}  %% Adjust vertical spacing at the top
\\begin{center}
    {\\LARGE \\textbf{Detalhes do Correspondente em %s}}\\\\  %% Title with country name
    \\vspace{0.5cm}
    {\\large Data: %s}  %% Date of the details
\\end{center}
\\section*{Informações do Correspondente}
\\begin{tabular}{|L{5cm}|L{9cm}|}
    \\hline
    \\rowcolor{readerred}
    \\color{white}\\textbf{Campo} & \\color{white}\\textbf{Valor} \\\\  %% Table header
    \\hline
    País de Atuação & %s \\\\  %% Country of operation
    Nome do Correspondente & %s \\\\  %% Correspondent name
    Última Atualização & %s \\\\  %% Last update date
    Peculiaridades de Trâmites & %s \\\\  %% Process peculiarities
    Descrição dos Documentos Necessários & %s \\\\  %% Description of required documents
    \\hline
\\end{tabular}
\\newpage
"""

### 🛠️ Placeholder Normalization & Debug Logging – Budget Tool for Trademarks

This segment ensures consistent formatting placeholders across all LaTeX templates. It:

- Replaces any erroneous uppercase `%S` with the correct lowercase `%s` in all predefined LaTeX template strings to maintain compatibility with Python's string formatting operations.
- Uses regular expressions to scan each template for `%s` placeholders, counting and printing the occurrences.
- Outputs debugging information to verify that placeholder replacement was successful and that all expected dynamic fields remain intact.

This quality control step mitigates runtime formatting errors and ensures templates are safely ready for downstream variable injection.


In [60]:
# Fix any %S placeholders
latex_template_deposito = latex_template_deposito.replace('%S', '%s')
latex_template_deposito_page = latex_template_deposito_page.replace('%S', '%s')
latex_template_prorrogacao_page = latex_template_prorrogacao_page.replace('%S', '%s')
latex_template_details = latex_template_details.replace('%S', '%s')
latex_template_details_page = latex_template_details_page.replace('%S', '%s')

# Debug: Verify placeholders
placeholders_deposito = re.findall(r'%s', latex_template_deposito)
print(f"Debug: Found {len(placeholders_deposito)} placeholders in latex_template_deposito: {placeholders_deposito}")
placeholders_deposito_page = re.findall(r'%s', latex_template_deposito_page)
print(f"Debug: Found {len(placeholders_deposito_page)} placeholders in latex_template_deposito_page: {placeholders_deposito_page}")
placeholders_prorrogacao_page = re.findall(r'%s', latex_template_prorrogacao_page)
print(f"Debug: Found {len(placeholders_prorrogacao_page)} placeholders in latex_template_prorrogacao_page: {placeholders_prorrogacao_page}")
placeholders_details = re.findall(r'%s', latex_template_details)
print(f"Debug: Found {len(placeholders_details)} placeholders in latex_template_details: {placeholders_details}")
placeholders_details_page = re.findall(r'%s', latex_template_details_page)
print(f"Debug: Found {len(placeholders_details_page)} placeholders in latex_template_details_page: {placeholders_details_page}")

Debug: Found 1 placeholders in latex_template_deposito: ['%s']
Debug: Found 6 placeholders in latex_template_deposito_page: ['%s', '%s', '%s', '%s', '%s', '%s']
Debug: Found 6 placeholders in latex_template_prorrogacao_page: ['%s', '%s', '%s', '%s', '%s', '%s']
Debug: Found 1 placeholders in latex_template_details: ['%s']
Debug: Found 7 placeholders in latex_template_details_page: ['%s', '%s', '%s', '%s', '%s', '%s', '%s']


### 🔤 Text Sanitization & Formatting Utilities – Budget Tool for Trademarks

This utility block provides key functions for data sanitization, localization, and structured display. It:

- Defines `escape_latex()`, which escapes LaTeX-sensitive characters (e.g., `&`, `%`, `_`, `$`, `#`, `|`) to prevent syntax errors in document rendering. Also handles missing values by returning `"N/A"`.
- Implements `format_brazilian()`, which converts numerical values into Brazilian currency format with comma as decimal separator and dot as thousand separator (e.g., `1.234,56`).
- Includes `display_countries()`, a protected utility for visualizing a list of countries in a neatly formatted multi-column DataFrame. It:
  - Splits the list evenly across up to five columns.
  - Adds itemized numbering for quick reference.
  - Outputs a console-rendered table for user-friendly validation.

These functions enforce data integrity, localization compliance, and readability for user-facing and LaTeX-bound outputs.


In [61]:
# Function to escape special LaTeX characters in text to prevent compilation errors
def escape_latex(text):
    """
    Escape special characters in text for safe inclusion in LaTeX documents.

    Parameters:
        text: Input text that may contain special characters.

    Returns:
        str: Text with special characters escaped for LaTeX.
    """
    if pd.isna(text):
        return "N/A"  # Handle NaN values from the spreadsheet
    return str(text).replace("&", "\\&").replace("%", "\\%").replace("#", "\\#").replace("_", "\\_").replace("$", "\\$").replace("|", "\\textbar")

# Function to format numbers in Brazilian style (e.g., 1.234,56)
def format_brazilian(value):
    """
    Format a numeric value into Brazilian currency style.

    Parameters:
        value: Numeric value to format.

    Returns:
        str: Formatted string with commas as decimal separators and dots as thousands separators.
    """
    if pd.isna(value):
        return "0,00"  # Default for NaN values
    return f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# Function to display available countries in a multi-column table
def display_countries(countries, num_columns=5):
    """
    Display a list of countries in a tabular format for user selection.

    Parameters:
        countries (list): List of country names.
        num_columns (int): Number of columns in the table (default is 5).
    """
    num_rows = math.ceil(len(countries) / num_columns)
    data = []
    for i in range(num_rows):
        row = []
        for j in range(num_columns):
            idx = i + j * num_rows
            if idx < len(countries):
                row.append(f"{idx + 1}. {countries[idx]}")
            else:
                row.append("")
        data.append(row)
    df = pd.DataFrame(data, columns=[f"Grupo {i+1}" for i in range(num_columns)])
    print("\nPaíses disponíveis:")
    print(df.to_string(index=False))

### 📝 Trademark Budget Data Collection – Core Interaction Module

This block implements the core interactive logic for collecting data required to budget a trademark operation (Depósito or Prorrogação). Key responsibilities:

### 🔄 `collect_marca_details(...)` – User-Guided Data Intake

- **Country & Correspondent Selection**:  
  Guided input to select a valid jurisdiction and correspondent from the dataset (`sheet_df`), with validation for user inputs. Country selection uses a multi-column console list (via `display_countries`).

- **Trademark Details Collection**:
  - **Name**: Captures and sanitizes user-provided trademark name using LaTeX escaping.
  - **Operation Type**: Determines if the quote is for a new deposit (`Depósito`) or a renewal (`Prorrogação`).
  - **Branching Logic**:
    - **For Depósitos**:
      - Collects the trademark type (`Mista`, `Nominativa`, or `Figurativa`).
      - Captures number of classes.
      - Generates corresponding labels for budgeting from known templates.
    - **For Prorrogações**:
      - Allows user to choose between *prazo ordinário* and *extraordinário*.
      - Gathers number of classes.
      - Maps selection to appropriate budget labels.

- **Returns**: A structured dictionary with normalized inputs:
  ```python
  {
      'selected_country': ...,
      'selected_correspondent': ...,
      'trademark_name': ...,
      'deposit_option': ...,
      'stored_labels': [...],
      'additional_class_labels': [...],
      'num_classes': ...
  }

### 🔎 'get_trademark_type(...)' – Type Inference Helper

- Scans the stored_labels returned from collect_marca_details to infer the trademark's type (Mista, Nominativa, Figurativa).

- Returns an empty string if type is not encoded in labels.

These functions constitute the primary user input workflow of the tool and directly support decision logic for dynamic budget generation based on jurisdictional and procedural parameters.

In [62]:

# Function to collect details for a single trademark budget from user input
def collect_marca_details(selected_country=None, selected_correspondent=None):
    """
    Collect information from the user about a trademark for budgeting.

    Parameters:
        selected_country (str, optional): Pre-selected country if provided.
        selected_correspondent (str, optional): Pre-selected correspondent if provided.

    Returns:
        dict or None: Dictionary with trademark details or None if user exits.
    """
    # Step 1: Ask for country if not provided
    if selected_country is None:
        while True:
            countries = sorted(sheet_df['Pais_de_Atuacao'].dropna().unique())
            display_countries(countries, num_columns=5)
            country_input = input("\nQual o país onde será feito o depósito? ").strip()
            if country_input.upper() == EXIT_KEYWORD:
                return None
            if country_input.isdigit() and 1 <= int(country_input) <= len(countries):
                selected_country = countries[int(country_input) - 1]
                break
            if country_input in countries:
                selected_country = country_input
                break
            print("Entrada inválida. Digite o número ou o nome do país.")

    # Step 2: Ask for correspondent if not provided
    if selected_correspondent is None:
        while True:
            print("\nCorrespondentes disponíveis:")
            correspondents = sorted(sheet_df[sheet_df['Pais_de_Atuacao'] == selected_country]['Nome_do_Correspondente'].dropna().unique())
            for i, corr in enumerate(correspondents, 1):
                print(f"{i}. {corr}")
            corr_input = input("Qual o correspondente com o qual irá orçar? ").strip()
            if corr_input.upper() == EXIT_KEYWORD:
                return None
            if corr_input.isdigit() and 1 <= int(corr_input) <= len(correspondents):
                selected_correspondent = correspondents[int(corr_input) - 1]
                break
            if corr_input in correspondents:
                selected_correspondent = corr_input
                break
            print("Entrada inválida. Digite o número ou o nome do correspondente.")

    # Step 3: Ask for trademark name
    trademark_name = input("Digite o nome da marca: ").strip()
    if trademark_name.upper() == EXIT_KEYWORD:
        return None
    trademark_name = escape_latex(trademark_name)  # Escape special characters for LaTeX

    # Step 4: Ask if it's a deposit or extension
    while True:
        print("\nEstá orçando um depósito de marca ou prorrogação?")
        deposit_options = ["Depósito", "Prorrogação"]
        for i, opt in enumerate(deposit_options, 1):
            print(f"{i}. {opt}")
        deposit_input = input().strip().capitalize()
        if deposit_input.upper() == EXIT_KEYWORD:
            return None
        if deposit_input.isdigit() and 1 <= int(deposit_input) <= len(deposit_options):
            deposit_option = deposit_options[int(deposit_input) - 1]
            break
        if deposit_input in deposit_options:
            deposit_option = deposit_input
            break
        print("Entrada inválida. Digite o número ou a opção (Depósito/Prorrogação).")

    # Step 5: Collect additional details based on deposit_option
    if deposit_option == "Depósito":
        # Ask for type of trademark
        while True:
            print("\nÉ um depósito de marca Mista, Nominativa ou Figurativa?")
            trademark_types = ["Mista", "Nominativa", "Figurativa"]
            for i, ttype in enumerate(trademark_types, 1):
                print(f"{i}. {ttype}")
            type_input = input().strip().capitalize()
            if type_input.upper() == EXIT_KEYWORD:
                return None
            if type_input.isdigit() and 1 <= int(type_input) <= len(trademark_types):
                deposit_type = trademark_types[int(type_input) - 1]
                break
            if type_input in trademark_types:
                deposit_type = type_input
                break
            print("Entrada inválida. Digite o número ou o tipo de marca.")

        # Ask for number of classes
        while True:
            class_input = input("A marca contém quantas classes? ").strip()
            if class_input.upper() == EXIT_KEYWORD:
                return None
            if class_input.isdigit():
                num_classes = int(class_input)
                break
            print("Entrada inválida. Digite um número.")

        # Define service labels for deposit
        stored_labels = [
            'Busca',
            'Valor_Associados_a_Documentos_de_Deposito_de_Marca',
            f'Deposito_de_Marca_{deposit_type}_1_Classe',
            'Concessao_de_Marca_1_Classe'
        ]
        additional_class_labels = []
        if num_classes > 1:
            additional_class_labels.append(f'Deposito_de_Marca_{deposit_type}_Classe_Adicional')
            additional_class_labels.append(f'Concessao_de_Marca_Classe_Adicional')
    else:
        # Ask which extension service is being budgeted
        services = [
            "Prorrogacao_Prazo_Ordinario",
            "Prorrogacao_Prazo_Extra_Ordinario"
        ]
        while True:
            print("\nQual serviço de prorrogação está sendo orçado?")
            for i, svc in enumerate(services, 1):
                print(f"{i}. {svc.replace('_', ' ')}")
            service_input = input().strip()
            if service_input.upper() == EXIT_KEYWORD:
                return None
            if service_input.isdigit() and 1 <= int(service_input) <= len(services):
                service_input = services[int(service_input) - 1]
                break
            if service_input in services or service_input.replace(" ", "_") in services:
                service_input = service_input.replace(" ", "_")
                break
            print("Entrada inválida. Digite o número ou o nome do serviço.")

        # Ask for number of classes
        while True:
            class_input = input("A marca contém quantas classes? ").strip()
            if class_input.upper() == EXIT_KEYWORD:
                return None
            if class_input.isdigit():
                num_classes = int(class_input)
                break
            print("Entrada inválida. Digite um número.")

        # Define service labels for extension
        stored_labels = [f'{service_input}_1_Classe']
        additional_class_labels = []
        if num_classes > 1:
            additional_class_labels.append(f'{service_input}_Classe_Adicional')

    # Return collected details as a dictionary
    return {
        'selected_country': selected_country,
        'selected_correspondent': selected_correspondent,
        'trademark_name': trademark_name,
        'deposit_option': deposit_option,
        'stored_labels': stored_labels,
        'additional_class_labels': additional_class_labels,
        'num_classes': num_classes
    }

# Function to determine the trademark type from stored labels
def get_trademark_type(stored_labels):
    """
    Extract the trademark type (Mista, Nominativa, Figurativa) from the list of service labels.

    Parameters:
        stored_labels (list): List of service labels associated with the trademark.

    Returns:
        str: The trademark type or an empty string if not found.
    """
    for label in stored_labels:
        if 'Deposito_de_Marca_Mista_1_Classe' in label:
            return 'Mista'
        elif 'Deposito_de_Marca_Nominativa_1_Classe' in label:
            return 'Nominativa'
        elif 'Deposito_de_Marca_Figurativa_1_Classe' in label:
            return 'Figurativa'
    return ''  # Default if no type is found

### 📄 LaTeX Budget Generator – `generate_marca_latex(...)`

This function automates LaTeX generation for trademark budgeting based on dynamic input parameters and pricing data. It synthesizes formatted outputs for both the **budget summary** and **procedural details** pages.

---

## 🔧 Function: `generate_marca_latex(marca_details, company_name)`

### 🎯 Purpose:
Generate LaTeX code blocks for:
- A comprehensive budget page
- A procedural information page
- Internal return structure for further processing

---

## 🔑 Inputs:
- `marca_details`: dictionary returned by `collect_marca_details(...)`, including:
  - Country, correspondent, trademark name
  - Deposit type (`Depósito` or `Prorrogação`)
  - Pricing labels (`stored_labels`, `additional_class_labels`)
  - Number of trademark classes
- `company_name`: name of the requesting entity

---

## ⚙️ Process Overview:

### 1. **Data Lookup**
- Fetches the first row of `sheet_df` matching the selected country and correspondent.
- Extracts base prices from relevant columns.

### 2. **Label Mapping**
- Dynamically categorizes input labels by purpose:
  - `busca_labels`, `document_labels`, `deposito_labels`, `concessao_labels`, `prorrogacao_labels`
  - Handles additional classes for each.

### 3. **Label Processing via `process_label(...)`**
Encapsulates logic to:
- Retrieve base cost from dataset
- Add honorários (from `honorarios_dict`)
- Apply tax rate (23.5%) unless exempt (e.g., document fees)
- Convert values to Brazilian currency format
- Append formatted rows to `latex_rows`
- Update cumulative totals

Totals tracked:
- `total_deposito`, `total_concessao`, `total_prorrogacao`, `total_honorarios`, `total_converted_sum`

### 4. **LaTeX Construction**
- Inserts line items per service
- Summarizes group totals depending on operation type:
  - `TOTAL PARA DEPÓSITO DE MARCA`
  - `TOTAL ESTIMADO PARA CONCESSÃO`
  - `TOTAL PARA PRORROGAÇÃO`
- Injects data into precompiled LaTeX templates:
  - `latex_template_deposito_page`
  - `latex_template_prorrogacao_page`
  - `latex_template_details_page`

---

## 📤 Returns:
- `budget_latex_page`: LaTeX string with table and cost summary
- `details_latex_page`: LaTeX string with procedural and document info
- A result `dict` containing:
  ```python
  {
      'marca_details': ...,
      'company_name': ...,
      'total_converted_sum': ...,
      'value_breakdown': ...,        # Per-label cost breakdown
      'stored_labels': [...],
      'total_deposito': ...,
      'total_concessao': ...
  }


In [63]:

# Function to generate LaTeX content and DOCX data for a single trademark budget
def generate_marca_latex(marca_details, company_name):
    """
    Generate LaTeX content for budget and details PDFs and collect data for DOCX generation.

    Parameters:
        marca_details (dict): Details of the trademark collected from user input.
        company_name (str): Name of the company (Titular).

    Returns:
        tuple: (budget LaTeX page, details LaTeX page, DOCX data dictionary).
    """
    global generated_docx_files  # Access global list
    selected_country = marca_details['selected_country']
    selected_correspondent = marca_details['selected_correspondent']
    trademark_name = marca_details['trademark_name']
    deposit_option = marca_details['deposit_option']
    stored_labels = marca_details['stored_labels']
    additional_class_labels = marca_details['additional_class_labels']
    num_classes = marca_details['num_classes']

    # Select the row from the spreadsheet matching the country and correspondent
    row = sheet_df[(sheet_df['Pais_de_Atuacao'] == selected_country) & (sheet_df['Nome_do_Correspondente'] == selected_correspondent)].iloc[0]

    # Initialize variables for cost calculation and LaTeX table generation
    value_breakdown = {}  # Store cost breakdown for each service
    latex_rows = []  # Store rows for the LaTeX table
    total_honorarios = 0  # Total including fees (not directly used in output)
    total_deposito = 0  # Total cost for deposit phase
    total_concessao = 0  # Total estimated cost for concession phase
    total_prorrogacao = 0  # Total cost for extension
    total_converted_sum = 0  # Grand total for all services
    processed_labels = set()  # Track processed labels to avoid duplicates

    # Categorize labels for ordered processing
    busca_labels = ['Busca'] if deposit_option == "Depósito" else []
    document_labels = ['Valor_Associados_a_Documentos_de_Deposito_de_Marca'] if deposit_option == "Depósito" else []
    deposito_labels = [label for label in stored_labels if "Deposito_de_Marca" in label]
    concessao_labels = [label for label in stored_labels if "Concessao_de_Marca" in label]
    prorrogacao_labels = [label for label in stored_labels if "Prorrogacao" in label]
    additional_deposito = [label for label in additional_class_labels if "Deposito_de_Marca" in label]
    additional_concessao = [label for label in additional_class_labels if "Concessao_de_Marca" in label]
    additional_prorrogacao = [label for label in additional_class_labels if "Prorrogacao" in label]

    # Helper function to process a service label and calculate costs
    def process_label(col, is_additional=False, multiplier=1):
        if col in row and pd.notnull(row[col]) and col not in processed_labels:
            # Extract correspondent fee and multiply by number of additional classes if applicable
            extracted_val = row[col] * multiplier if pd.notnull(row[col]) else 0
            # Get professional fee from dictionary
            honor_key = f"Honorarios_{col}"
            honor_val = honorarios_dict.get(honor_key, 0) * multiplier
            # Calculate total value with a 23.5% markup (except for document fees)
            total_val = (extracted_val + honor_val) * (1.235 if col not in ['Valor_Associados_a_Documentos_de_Deposito_de_Marca'] else 1)
            total_converted = total_val  # No currency conversion applied here
            # Store breakdown for DOCX generation
            value_breakdown[col] = {
                'extracted_val': extracted_val,
                'honor_val': honor_val,
                'total_val': total_val,
                'total_converted': total_converted
            }
            nonlocal total_honorarios, total_converted_sum, total_deposito, total_concessao, total_prorrogacao
            total_honorarios += total_val
            total_converted_sum += total_converted
            # Categorize totals based on service type
            if deposit_option == "Depósito" and "Concessao" not in col and col not in busca_labels:
                total_deposito += total_converted
            elif deposit_option == "Depósito" and "Concessao" in col:
                total_concessao += total_converted
            else:
                total_prorrogacao += total_converted
            # Format description for LaTeX table
            description = col.replace('_', ' ').replace("Deposito", "Depósito").replace("Concessao", "Concessão").replace("Prorrogacao", "Prorrogação")
            valor_original = format_brazilian(extracted_val)
            total_converted_str = format_brazilian(total_converted)
            prefix = "\\textbf" if col in busca_labels or not is_additional else ""
            latex_rows.append(f"    {prefix}{{{description}}} & {valor_original} & {total_converted_str} \\\\ \\hline")
            processed_labels.add(col)

    # Process all labels in order
    for col in busca_labels:
        process_label(col)
    for col in document_labels:
        process_label(col)
    for col in deposito_labels:
        process_label(col)
    if num_classes > 1:
        additional_classes = num_classes - 1
        for col in additional_deposito:
            process_label(col, is_additional=True, multiplier=additional_classes)
    # Add subtotal for deposit phase
    if deposit_option == "Depósito":
        value_breakdown["TOTAL_PARA_DEPOSITO"] = {'total_converted': total_deposito}
        latex_rows.append(f"    \\textbf{{TOTAL PARA DEPÓSITO DE MARCA}} & & \\textbf{{{format_brazilian(total_deposito)}}} \\\\ \\hline")
    for col in concessao_labels:
        process_label(col)
    if deposit_option == "Depósito" and num_classes > 1:
        additional_classes = num_classes - 1
        for col in additional_concessao:
            process_label(col, is_additional=True, multiplier=additional_classes)
    # Add subtotal for concession phase
    if deposit_option == "Depósito":
        value_breakdown["TOTAL_ESTIMADO_PARA_CONCESSAO"] = {'total_converted': total_concessao}
        latex_rows.append(f"    \\textbf{{TOTAL ESTIMADO PARA CONCESSÃO}} & & \\textbf{{{format_brazilian(total_concessao)}}} \\\\ \\hline")
    for col in prorrogacao_labels:
        process_label(col)
    if num_classes > 1:
        additional_classes = num_classes - 1
        for col in additional_prorrogacao:
            process_label(col, is_additional=True, multiplier=additional_classes)
    # Add subtotal for extension
    if deposit_option != "Depósito":
        value_breakdown["TOTAL_PARA_PRORROGACAO"] = {'total_converted': total_prorrogacao}
        latex_rows.append(f"    \\textbf{{TOTAL PARA PRORROGAÇÃO}} & & \\textbf{{{format_brazilian(total_prorrogacao)}}} \\\\ \\hline")

    # Join table rows and format the budget page
    latex_table = "\n".join(latex_rows)
    total_converted_str = format_brazilian(total_converted_sum)
    latex_page_template = latex_template_deposito_page if deposit_option == "Depósito" else latex_template_prorrogacao_page
    budget_latex_page = latex_page_template % (selected_country, formatted_date, company_name, trademark_name, latex_table, total_converted_str)

    # Generate details page with correspondent information
    details = {
        'pais_de_atuacao': escape_latex(row['Pais_de_Atuacao']),
        'nome_do_correspondente': escape_latex(row['Nome_do_Correspondente']),
        'ultima_atualizacao': escape_latex(row['Ultima_Atualizacao']),
        'peculiaridades_de_tramites': escape_latex(row['Peculiaridades_de_Tramites']),
        'descricao_dos_documentos_necessarios': escape_latex(row['Descricao_dos_Documentos_Necessarios'])
    }
    details_latex_page = latex_template_details_page % (
        selected_country, formatted_date, details['pais_de_atuacao'], details['nome_do_correspondente'],
        details['ultima_atualizacao'], details['peculiaridades_de_tramites'], details['descricao_dos_documentos_necessarios']
    )

    # Debug: Display cost breakdown for verification
    print(f"Debug: value_breakdown for {trademark_name} in {selected_country}: {value_breakdown}")

    # Return LaTeX pages and data for DOCX generation
    return budget_latex_page, details_latex_page, {
        'marca_details': marca_details,
        'company_name': company_name,
        'total_converted_sum': total_converted_sum,
        'value_breakdown': value_breakdown,
        'stored_labels': stored_labels,
        'total_deposito': total_deposito,
        'total_concessao': total_concessao
    }

# Reinitialize global list to track DOCX files (redundant declaration, but kept as per original code)
generated_docx_files = []

### 📄 DOCX Budget Generator – `generate_docx_from_template(...)`

This function generates a DOCX file for trademark budgeting by populating a template with dynamic data. It replaces placeholders in paragraphs and tables with formatted values for budget details.

---

## 🔧 Function: `generate_docx_from_template(marca_details, company_name, total_converted_sum, value_breakdown, stored_labels, total_deposito, total_concessao)`

### 🎯 Purpose:
- Populate a DOCX template with trademark budget data
- Generate a formatted budget document with country, company, and cost details
- Save the output with a structured filename

---

## 🔑 Inputs:
- `marca_details`: dictionary containing:
  - `selected_country`: Country of trademark registration
  - `trademark_name`: Name of the trademark
- `company_name`: Name of the requesting entity
- `total_converted_sum`: Total cost in Brazilian currency
- `value_breakdown`: Dictionary with per-service cost details
- `stored_labels`: List of pricing labels (e.g., `Busca`, `Deposito_de_Marca`)
- `total_deposito`: Total cost for trademark deposit
- `total_concessao`: Total estimated cost for trademark concession

---

## ⚙️ Process Overview:

### 1. **Template Loading**
- Loads a DOCX template from `DOCX_TEMPLATE_PATH` using `python-docx`.

### 2. **Helper Function: `get_value(...)`**
- Retrieves values from `value_breakdown` for a given service and key, with a default of 0.

### 3. **Depósito Label Determination**
- Identifies trademark type (`Mista`, `Nominativa`, or generic) from `stored_labels`.
- Selects the appropriate `deposito_label` based on trademark type and class (e.g., `Deposito_de_Marca_Mista_1_Classe`).
- Logs the selected label and its value for debugging.

### 4. **Placeholder Replacement**
- Iterates through paragraphs and table cells in the DOCX template.
- Replaces placeholders with formatted values, including:
  - `{{ pais }}`: Country
  - `{{ data }}`: Formatted date
  - `{{ titular }}`: Company name
  - `{{ marca }}`: Trademark name
  - `{{ tipo_marca }}`: Trademark type
  - `{{ taxa_busca_usd }}`, `{{ total_busca_usd }}`: Search fees
  - `{{ taxa_documentos_usd }}`, `{{ total_documentos_usd }}`: Document fees
  - `{{ taxa_deposito_usd }}`, `{{ total_deposito_usd }}`: Deposit fees
  - `{{ taxa_concessao_usd }}`, `{{ total_concessao_usd }}`: Concession fees
  - `{{ total_para_deposito_usd }}`: Total deposit cost
  - `{{ total_estimado_concessao_usd }}`: Total estimated concession cost
  - `{{ total_final_usd }}`: Total final cost
- Uses `format_brazilian(...)` for currency formatting.

### 5. **File Naming and Saving**
- Sanitizes `trademark_name` and `selected_country` for safe filename generation.
- Constructs filename: `Orçamento_{trademark_name}_{trademark_type}_{country}_{date}.docx`.
- Saves the populated DOCX to `output_dir`.
- Logs the output path.

---

## 📤 Returns:
- None (saves DOCX file to disk)
- Prints confirmation of successful file generation



In [64]:

# Function to generate a DOCX file for a single-class trademark deposit
def generate_docx_from_template(marca_details, company_name, total_converted_sum, value_breakdown, stored_labels, total_deposito, total_concessao):
    """
    Generate a DOCX file for a single-class trademark deposit using a template.

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total estimated cost.
        value_breakdown (dict): Cost breakdown for each service.
        stored_labels (list): List of service labels.
        total_deposito (float): Total cost for deposit phase.
        total_concessao (float): Total estimated cost for concession phase.
    """
    global generated_docx_files  # Access global list to track generated files
    doc = Document(DOCX_TEMPLATE_PATH)  # Load the single-class deposit template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Determine the specific deposit label based on trademark type
    trademark_type = get_trademark_type(stored_labels).lower()
    deposito_keyword = "Mista" if "mista" in trademark_type else "Nominativa" if "nominativa" in trademark_type else ""
    deposito_label = next(
        (label for label in stored_labels if f"Deposito_de_Marca_{deposito_keyword}_1_Classe" in label),
        next((label for label in stored_labels if "Deposito_de_Marca" in label), None)
    )
    # Debug: Confirm the selected deposit label and its value
    print(f"Debug: Selected deposito_label for {marca_details['trademark_name']}: {deposito_label}, value: {get_value(deposito_label, 'total_converted')}")

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_busca_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_busca_usd }}', format_brazilian(get_value('Busca', 'extracted_val')))
        if '{{ total_busca_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_busca_usd }}', format_brazilian(get_value('Busca', 'total_converted')))
        if '{{ taxa_documentos_usd }}' in paragraph.text:
            document_fee = get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'extracted_val')
            paragraph.text = paragraph.text.replace('{{ taxa_documentos_usd }}', format_brazilian(document_fee)).replace('{{ taxa_documentos_usd }} ', '')
        if '{{ total_documentos_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_documentos_usd }}', format_brazilian(get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'total_converted')))
        if '{{ tipo_marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ tipo_marca }}', get_trademark_type(stored_labels))
        if '{{ taxa_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_deposito_usd }}', format_brazilian(get_value(deposito_label, 'extracted_val')))
        if '{{ total_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_deposito_usd }}', format_brazilian(get_value(deposito_label, 'total_converted')))
        if '{{ total_para_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_deposito_usd }}', format_brazilian(total_deposito))
        if '{{ taxa_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'extracted_val')))
        if '{{ total_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'total_converted')))
        if '{{ total_estimado_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_estimado_concessao_usd }}', format_brazilian(total_concessao))
        if '{{ total_final_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_busca_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_busca_usd }}', format_brazilian(get_value('Busca', 'extracted_val')))
                    if '{{ total_busca_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_busca_usd }}', format_brazilian(get_value('Busca', 'total_converted')))
                    if '{{ taxa_documentos_usd }}' in paragraph.text:
                        document_fee = get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'extracted_val')
                        paragraph.text = paragraph.text.replace('{{ taxa_documentos_usd }}', format_brazilian(document_fee)).replace('{{ taxa_documentos_usd }} ', '')
                    if '{{ total_documentos_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_documentos_usd }}', format_brazilian(get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'total_converted')))
                    if '{{ tipo_marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ tipo_marca }}', get_trademark_type(stored_labels))
                    if '{{ taxa_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_deposito_usd }}', format_brazilian(get_value(deposito_label, 'extracted_val')))
                    if '{{ total_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_deposito_usd }}', format_brazilian(get_value(deposito_label, 'total_converted')))
                    if '{{ total_para_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_deposito_usd }}', format_brazilian(total_deposito))
                    if '{{ taxa_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'extracted_val')))
                    if '{{ total_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'total_converted')))
                    if '{{ total_estimado_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_estimado_concessao_usd }}', format_brazilian(total_concessao))
                    if '{{ total_final_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    trademark_type = get_trademark_type(stored_labels)
    docx_filename = f"Orçamento_{safe_trademark_name}_{trademark_type}_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file and track it
    doc.save(docx_output_path)
    generated_docx_files.append(docx_filename)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

### 📄 DOCX Generator for Multi-Class Trademark Budget – `generate_docx_from_multi_class_template(...)`

This function generates a DOCX file for a trademark budget using a multi-class template, filling in details based on the provided parameters.

---

## 🔧 Function: `generate_docx_from_multi_class_template(marca_details, company_name, total_converted_sum, value_breakdown, stored_labels, total_deposito, total_concessao)`

### 🎯 Purpose:
Generate a DOCX file for a trademark budget, populating a template with details about the trademark, company, and financial breakdowns for deposit and concession phases.

---

## 🔑 Inputs:
- `marca_details`: Dictionary containing trademark details, including:
  - `selected_country`: The country for the trademark registration.
  - `trademark_name`: The name of the trademark.
  - `additional_class_labels`: Labels for additional classes, if applicable.
- `company_name`: The name of the company requesting the budget.
- `total_converted_sum`: The total sum of all costs, converted to the desired currency.
- `value_breakdown`: A dictionary with cost breakdowns for each service, structured as `{service: {'extracted_val': value, 'total_converted': value}}`.
- `stored_labels`: A list of labels corresponding to different services or cost items.
- `total_deposito`: The total cost for the deposit phase.
- `total_concessao`: The total estimated cost for the concession phase.

---

## ⚙️ Process Overview:

### 1. **Load Template**
- Loads the DOCX template from `DOCX_MULTI_CLASS_TEMPLATE_PATH`.

### 2. **Helper Function**
- Defines `get_value(service, key, default=0)` to retrieve values from `value_breakdown`. Returns the specified key's value for the given service, or the default if not found.

### 3. **Determine Trademark Type and Deposit Label**
- Determines the trademark type (e.g., "Mista" or "Nominativa") using `get_trademark_type(stored_labels)`.
- Selects the appropriate deposit label from `stored_labels` based on the trademark type, prioritizing specific matches like `Deposito_de_Marca_Mista_1_Classe` or falling back to a general `Deposito_de_Marca` label.
- Prints a debug message with the selected deposit label and its value.

### 4. **Replace Placeholders in Paragraphs**
- Iterates through all paragraphs in the document, replacing placeholders such as:
  - `{{ pais }}` with `marca_details['selected_country']`.
  - `{{ data }}` with `formatted_date`.
  - `{{ titular }}` with `company_name`.
  - `{{ marca }}` with `marca_details['trademark_name']`.
  - Financial placeholders (e.g., `{{ taxa_busca_usd }}`, `{{ total_deposito_usd }}`) with values from `value_breakdown`, formatted using `format_brazilian`.
  - Additional class costs from `marca_details['additional_class_labels']` for deposit and concession phases.

### 5. **Replace Placeholders in Tables**
- Iterates through all tables, applying the same placeholder replacements in table cells as in paragraphs.

### 6. **Generate and Save File**
- Sanitizes the trademark name and country using `re.sub` to remove non-word characters.
- Constructs a filename in the format `Orçamento_<trademark>_<type>_<country>_<date>.docx`.
- Saves the modified document to `output_dir` and prints a success message with the file path.

---

## 📤 Returns:
- This function does not return any value. It saves the generated DOCX file to the specified output path and prints a confirmation message.

---

**Notes:**
- Assumes a template at `DOCX_MULTI_CLASS_TEMPLATE_PATH` with matching placeholders.
- Relies on external functions: `get_trademark_type`, `format_brazilian`, and variables: `formatted_date`, `filename_date`, `output_dir`.
- Uses `re` and `os` modules for filename sanitization and path handling.


In [65]:
# Function to generate a DOCX file for a multi-class trademark deposit
def generate_docx_from_multi_class_template(marca_details, company_name, total_converted_sum, value_breakdown, stored_labels, total_deposito, total_concessao):
    """
    Generate a DOCX file for a multi-class trademark deposit using a template.

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total estimated cost.
        value_breakdown (dict): Cost breakdown for each service.
        stored_labels (list): List of service labels.
        total_deposito (float): Total cost for deposit phase.
        total_concessao (float): Total estimated cost for concession phase.
    """
    global generated_docx_files  # Access global list to track generated files
    doc = Document(DOCX_MULTI_CLASS_TEMPLATE_PATH)  # Load the multi-class deposit template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Determine the specific deposit label based on trademark type
    trademark_type = get_trademark_type(stored_labels).lower()
    deposito_keyword = "Mista" if "mista" in trademark_type else "Nominativa" if "nominativa" in trademark_type else ""
    deposito_label = next(
        (label for label in stored_labels if f"Deposito_de_Marca_{deposito_keyword}_1_Classe" in label),
        next((label for label in stored_labels if "Deposito_de_Marca" in label), None)
    )
    # Debug: Confirm the selected deposit label and its value
    print(f"Debug: Selected deposito_label for {marca_details['trademark_name']}: {deposito_label}, value: {get_value(deposito_label, 'total_converted')}")

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_busca_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_busca_usd }}', format_brazilian(get_value('Busca', 'extracted_val')))
        if '{{ total_busca_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_busca_usd }}', format_brazilian(get_value('Busca', 'total_converted')))
        if '{{ taxa_documentos_usd }}' in paragraph.text:
            document_fee = get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'extracted_val')
            paragraph.text = paragraph.text.replace('{{ taxa_documentos_usd }}', format_brazilian(document_fee)).replace('{{ taxa_documentos_usd }} ', '')
        if '{{ total_documentos_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_documentos_usd }}', format_brazilian(get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'total_converted')))
        if '{{ tipo_marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ tipo_marca }}', get_trademark_type(stored_labels))
        if '{{ taxa_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_deposito_usd }}', format_brazilian(get_value(deposito_label, 'extracted_val')))
        if '{{ total_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_deposito_usd }}', format_brazilian(get_value(deposito_label, 'total_converted')))
        if '{{ total_para_deposito_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_deposito_usd }}', format_brazilian(total_deposito))
        if '{{ taxa_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ taxa_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'extracted_val')))
        if '{{ total_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'total_converted')))
        if '{{ total_estimado_concessao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_estimado_concessao_usd }}', format_brazilian(total_concessao))
        if '{{ total_final_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))
        if '{{ taxa_deposito_classe_adicional_usd }}' in paragraph.text:
            additional_deposito_label = next((label for label in marca_details['additional_class_labels'] if "Deposito_de_Marca" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_deposito_classe_adicional_usd }}', format_brazilian(get_value(additional_deposito_label, 'extracted_val')))
        if '{{ total_deposito_classe_adicional_usd }}' in paragraph.text:
            additional_deposito_label = next((label for label in marca_details['additional_class_labels'] if "Deposito_de_Marca" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_deposito_classe_adicional_usd }}', format_brazilian(get_value(additional_deposito_label, 'total_converted')))
        if '{{ taxa_concessao_classe_adicional_usd }}' in paragraph.text:
            additional_concessao_label = next((label for label in marca_details['additional_class_labels'] if "Concessao_de_Marca" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_concessao_classe_adicional_usd }}', format_brazilian(get_value(additional_concessao_label, 'extracted_val')))
        if '{{ total_concessao_classe_adicional_usd }}' in paragraph.text:
            additional_concessao_label = next((label for label in marca_details['additional_class_labels'] if "Concessao_de_Marca" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_concessao_classe_adicional_usd }}', format_brazilian(get_value(additional_concessao_label, 'total_converted')))
        if '{{ total_estimado_c_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_estimado_c_usd }}', format_brazilian(total_concessao))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_busca_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_busca_usd }}', format_brazilian(get_value('Busca', 'extracted_val')))
                    if '{{ total_busca_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_busca_usd }}', format_brazilian(get_value('Busca', 'total_converted')))
                    if '{{ taxa_documentos_usd }}' in paragraph.text:
                        document_fee = get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'extracted_val')
                        paragraph.text = paragraph.text.replace('{{ taxa_documentos_usd }}', format_brazilian(document_fee)).replace('{{ taxa_documentos_usd }} ', '')
                    if '{{ total_documentos_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_documentos_usd }}', format_brazilian(get_value('Valor_Associados_a_Documentos_de_Deposito_de_Marca', 'total_converted')))
                    if '{{ tipo_marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ tipo_marca }}', get_trademark_type(stored_labels))
                    if '{{ taxa_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_deposito_usd }}', format_brazilian(get_value(deposito_label, 'extracted_val')))
                    if '{{ total_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_deposito_usd }}', format_brazilian(get_value(deposito_label, 'total_converted')))
                    if '{{ total_para_deposito_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_deposito_usd }}', format_brazilian(total_deposito))
                    if '{{ taxa_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ taxa_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'extracted_val')))
                    if '{{ total_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_concessao_usd }}', format_brazilian(get_value('Concessao_de_Marca_1_Classe', 'total_converted')))
                    if '{{ total_estimado_concessao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_estimado_concessao_usd }}', format_brazilian(total_concessao))
                    if '{{ total_final_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))
                    if '{{ taxa_deposito_classe_adicional_usd }}' in paragraph.text:
                        additional_deposito_label = next((label for label in marca_details['additional_class_labels'] if "Deposito_de_Marca" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_deposito_classe_adicional_usd }}', format_brazilian(get_value(additional_deposito_label, 'extracted_val')))
                    if '{{ total_deposito_classe_adicional_usd }}' in paragraph.text:
                        additional_deposito_label = next((label for label in marca_details['additional_class_labels'] if "Deposito_de_Marca" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_deposito_classe_adicional_usd }}', format_brazilian(get_value(additional_deposito_label, 'total_converted')))
                    if '{{ taxa_concessao_classe_adicional_usd }}' in paragraph.text:
                        additional_concessao_label = next((label for label in marca_details['additional_class_labels'] if "Concessao_de_Marca" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_concessao_classe_adicional_usd }}', format_brazilian(get_value(additional_concessao_label, 'extracted_val')))
                    if '{{ total_concessao_classe_adicional_usd }}' in paragraph.text:
                        additional_concessao_label = next((label for label in marca_details['additional_class_labels'] if "Concessao_de_Marca" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_concessao_classe_adicional_usd }}', format_brazilian(get_value(additional_concessao_label, 'total_converted')))
                    if '{{ total_estimado_c_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_estimado_c_usd }}', format_brazilian(total_concessao))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    trademark_type = get_trademark_type(stored_labels)
    docx_filename = f"Orçamento_{safe_trademark_name}_{trademark_type}_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file and track it
    doc.save(docx_output_path)
    generated_docx_files.append(docx_filename)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

### 📄 DOCX Generator Functions for Trademark Prorogation Ordinary Term Budgets

These functions generate DOCX files for trademark prorogation budgets using predefined templates, replacing placeholders with provided data for single-class and multi-class scenarios.

---

## 🔧 Function: `generate_docx_from_prorrogacao_ordinario_template(marca_details, company_name, total_converted_sum, value_breakdown)`

### 🎯 Purpose
Generates a DOCX file for a **single-class** trademark prorogation budget, populating a template with trademark details, company information, and financial data.

---

### 🔑 Inputs
- **`marca_details`**: Dictionary containing trademark details:
  - `'selected_country'`: The country of the trademark prorogation.
  - `'trademark_name'`: The name of the trademark.
  - `'stored_labels'`: List of labels for services/costs, used to identify prorogation-specific values.
- **`company_name`**: String representing the name of the company requesting the budget.
- **`total_converted_sum`**: Numeric total cost for the prorogation, in converted currency.
- **`value_breakdown`**: Dictionary with cost breakdowns, structured as `{service: {'extracted_val': value, 'total_converted': value}}`.

---

### ⚙️ Process Overview

1. **Load Template**
   - Opens the DOCX template from `DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH` using `Document`.

2. **Define Helper Function**
   - `get_value(service, key, default=0)`: Retrieves values from `value_breakdown` for a given service and key, returning `default` if not found.

3. **Replace Placeholders in Paragraphs**
   - Iterates through all paragraphs, replacing:
     - `{{ pais }}` with `marca_details['selected_country']`.
     - `{{ data }}` with `formatted_date`.
     - `{{ titular }}` with `company_name`.
     - `{{ marca }}` with `marca_details['trademark_name']`.
     - `{{ taxa_prorrogacao_ordinario_usd }}` with the `extracted_val` from `value_breakdown`, formatted via `format_brazilian`, using a label from `stored_labels` containing `"Prorrogacao_Prazo_Ordinario"`.
     - `{{ total_prorrogacao_ordinario_usd }}` with the `total_converted` from `value_breakdown`, formatted similarly.
     - `{{ total_para_prorrogacao_ordinario_usd }}` with `total_converted_sum`, formatted via `format_brazilian`.

4. **Replace Placeholders in Tables**
   - Iterates through all tables, rows, and cells, applying the same replacements as in paragraphs.

5. **Save the Document**
   - Sanitizes `trademark_name` and `selected_country` with `re.sub` to remove non-word characters (except hyphens).
   - Constructs filename: `Orçamento_<trademark>_Prorrogacao_Ordinario_<country>_<filename_date>.docx`.
   - Saves to `output_dir` using `os.path.join`.
   - Prints a success message with the file path.

---

### 📤 Returns
- None. Saves the DOCX file and prints a confirmation message.

---

### 📝 Notes
- Requires `Document` (from `docx`), `re`, and `os` modules.
- Depends on external variables: `DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH`, `formatted_date`, `filename_date`, `output_dir`.
- Uses `format_brazilian` for currency formatting.
- Assumes `stored_labels` contains a `"Prorrogacao_Prazo_Ordinario"` label.

---

## 🔧 Function: `generate_docx_from_prorrogacao_ordinario_multi_class_template(marca_details, company_name, total_converted_sum, value_breakdown)`

### 🎯 Purpose
Generates a DOCX file for a **multi-class** trademark prorogation budget, extending the single-class functionality to include additional class costs.

---

### 🔑 Inputs
- **`marca_details`**: Dictionary containing trademark details:
  - `'selected_country'`: The country of the trademark prorogation.
  - `'trademark_name'`: The name of the trademark.
  - `'stored_labels'`: List of labels for main class services/costs.
  - `'additional_class_labels'`: List of labels for additional class services/costs.
- **`company_name`**: String representing the name of the company requesting the budget.
- **`total_converted_sum`**: Numeric total cost, including additional classes, in converted currency.
- **`value_breakdown`**: Dictionary with cost breakdowns, structured as `{service: {'extracted_val': value, 'total_converted': value}}`.

---

### ⚙️ Process Overview

1. **Load Template**
   - Opens the DOCX template from `DOCX_PRORROGACAO_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH` using `Document`.

2. **Define Helper Function**
   - `get_value(service, key, default=0)`: Retrieves values from `value_breakdown` for a given service and key, returning `default` if not found.

3. **Replace Placeholders in Paragraphs**
   - Iterates through all paragraphs, replacing:
     - `{{ pais }}` with `marca_details['selected_country']`.
     - `{{ data }}` with `formatted_date`.
     - `{{ titular }}` with `company_name`.
     - `{{ marca }}` with `marca_details['trademark_name']`.
     - `{{ taxa_prorrogacao_ordinario_usd }}` with the `extracted_val` from `value_breakdown` for the main class, formatted via `format_brazilian`, using a label from `stored_labels`.
     - `{{ total_prorrogacao_ordinario_usd }}` with the `total_converted` from `value_breakdown` for the main class, formatted similarly.
     - `{{ taxa_prorrogacao_classe_adicional_ordinario_usd }}` with the `extracted_val` from `value_breakdown` for additional classes, formatted via `format_brazilian`, using a label from `additional_class_labels`.
     - `{{ total_prorrogacao_classe_adicional_ordinario_usd }}` with the `total_converted` from `value_breakdown` for additional classes, formatted similarly.
     - `{{ total_para_prorrogacao_ordinario_usd }}` with `total_converted_sum`, formatted via `format_brazilian`.

4. **Replace Placeholders in Tables**
   - Iterates through all tables, rows, and cells, applying the same replacements as in paragraphs.

5. **Save the Document**
   - Sanitizes `trademark_name` and `selected_country` with `re.sub` to remove non-word characters (except hyphens).
   - Constructs filename: `Orçamento_<trademark>_Prorrogacao_Ordinario_<country>_<filename_date>.docx`.
   - Saves to `output_dir` using `os.path.join`.
   - Prints a success message with the file path.

---

### 📤 Returns
- None. Saves the DOCX file and prints a confirmation message.

---

### 📝 Notes
- Requires `Document` (from `docx`), `re`, and `os` modules.
- Depends on external variables: `DOCX_PRORROGACAO_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH`, `formatted_date`, `filename_date`, `output_dir`.
- Uses `format_brazilian` for currency formatting.
- Assumes `stored_labels` and `additional_class_labels` contain `"Prorrogacao_Prazo_Ordinario"` labels for main and additional classes, respectively.



In [66]:
# Function to generate a DOCX file for a single-class ordinary extension (first definition, appears redundant)
def generate_docx_from_prorrogacao_ordinario_template(marca_details, company_name, total_converted_sum, value_breakdown, stored_labels, total_prorrogacao):
    """
    Generate a DOCX file for a single-class ordinary extension using a template (first definition).

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total cost.
        value_breakdown (dict): Cost breakdown for each service.
        stored_labels (list): List of service labels.
        total_prorrogacao (float): Total cost for the extension.
    """
    global generated_docx_files  # Access global list to track generated files
    doc = Document(DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH)  # Load the single-class ordinary extension template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_prorrogacao_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in stored_labels if "Prorrogacao" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_usd }}', format_brazilian(total_prorrogacao))
        if '{{ total_final_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_prorrogacao_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in stored_labels if "Prorrogacao" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_usd }}', format_brazilian(total_prorrogacao))
                    if '{{ total_final_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_final_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    docx_filename = f"Orçamento_{safe_trademark_name}_Prorrogacao_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file and track it
    doc.save(docx_output_path)
    generated_docx_files.append(docx_filename)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

# Function to generate a DOCX file for a single-class ordinary extension (second definition)
def generate_docx_from_prorrogacao_ordinario_template(marca_details, company_name, total_converted_sum, value_breakdown):
    """
    Generate a DOCX file for a single-class ordinary extension using a template (second definition).

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total cost.
        value_breakdown (dict): Cost breakdown for each service.
    """
    doc = Document(DOCX_PRORROGACAO_ORDINARIO_TEMPLATE_PATH)  # Load the single-class ordinary extension template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_prorrogacao_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
        if '{{ total_para_prorrogacao_ordinario_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_prorrogacao_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
                    if '{{ total_para_prorrogacao_ordinario_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    docx_filename = f"Orçamento_{safe_trademark_name}_Prorrogacao_Ordinario_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file
    doc.save(docx_output_path)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

# Function to generate a DOCX file for a multi-class ordinary extension
def generate_docx_from_prorrogacao_ordinario_multi_class_template(marca_details, company_name, total_converted_sum, value_breakdown):
    """
    Generate a DOCX file for a multi-class ordinary extension using a template.

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total cost.
        value_breakdown (dict): Cost breakdown for each service.
    """
    doc = Document(DOCX_PRORROGACAO_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH)  # Load the multi-class ordinary extension template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_prorrogacao_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
        if '{{ taxa_prorrogacao_classe_adicional_ordinario_usd }}' in paragraph.text:
            additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_classe_adicional_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_classe_adicional_ordinario_usd }}' in paragraph.text:
            additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_classe_adicional_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'total_converted')))
        if '{{ total_para_prorrogacao_ordinario_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_prorrogacao_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
                    if '{{ taxa_prorrogacao_classe_adicional_ordinario_usd }}' in paragraph.text:
                        additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_classe_adicional_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_classe_adicional_ordinario_usd }}' in paragraph.text:
                        additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_classe_adicional_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'total_converted')))
                    if '{{ total_para_prorrogacao_ordinario_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    docx_filename = f"Orçamento_{safe_trademark_name}_Prorrogacao_Ordinario_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file
    doc.save(docx_output_path)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

### 📄 DOCX Generator Functions for Trademark Prorogation Extra Ordinary Term Budgets

These functions automate the generation of Word documents (.docx) using predefined templates tailored for **Extraordinary Renewal of Trademarks** – either single-class or multi-class cases. The scripts dynamically populate placeholders within DOCX files based on structured data inputs.

---

## 🔧 Function 1: `generate_docx_from_prorrogacao_extra_ordinario_template(...)`

### 🎯 Purpose:
Populate and export a DOCX template for **single-class extraordinary renewals**, using Jinja-style variable tags inside the Word document.

---

## 🔧 Function 2: `generate_docx_from_prorrogacao_extra_ordinario_multi_class_template(...)`

### 🎯 Purpose:
Same logic, adapted for **multi-class** extraordinary renewal cases, with additional handling for `additional_class_labels`.

---

## 🔑 Inputs:
- `marca_details`: Dictionary with:
  - `selected_country`
  - `trademark_name`
  - `stored_labels`
  - `additional_class_labels` (used only in the multi-class version)
- `company_name`: The client's name (titular)
- `total_converted_sum`: Final cost estimate in BRL or USD
- `value_breakdown`: Dictionary with extracted fees and totals, indexed by service name

---

## ⚙️ Process Overview:

### 1. **Template Loading**
- Opens a DOCX file from a pre-set path (`*_TEMPLATE_PATH`)
- Wraps logic in a Python-Docx `Document(...)` object

### 2. **Value Retrieval**
- Defines helper: `get_value(service, key, default=0)` for safe extraction of nested values from `value_breakdown`

### 3. **Text Replacement in Paragraphs**
Replaces placeholders like:
- `{{ pais }}` → `marca_details['selected_country']`
- `{{ marca }}` → `marca_details['trademark_name']`
- `{{ titular }}` → `company_name`
- `{{ data }}` → `formatted_date`
- `{{ taxa_... }}` and `{{ total_... }}` → pulled from `value_breakdown` using corresponding `label`

### 4. **Text Replacement in Tables**
- Performs the same substitution routine within all tables and cell paragraphs
- Includes additional handling for:
  - `{{ taxa_prorrogacao_classe_adicional_extra_ordinario_usd }}`
  - `{{ total_prorrogacao_classe_adicional_extra_ordinario_usd }}`

### 5. **Filename Construction & Export**
- Constructs filename based on cleaned country and trademark names
- Saves document to `output_dir`
- Prints success message

---

## 📤 Output:
- A fully generated `.docx` file saved to:

---

## ⚠️ Notes:
- Templates must contain all expected `{{ }}` placeholders for proper substitution
- Assumes `format_brazilian(...)` converts and formats numeric values to localized strings
- `filename_date` and `formatted_date` should be defined in outer scope prior to function call




In [67]:
# Function to generate a DOCX file for a single-class extraordinary extension
def generate_docx_from_prorrogacao_extra_ordinario_template(marca_details, company_name, total_converted_sum, value_breakdown):
    """
    Generate a DOCX file for a single-class extraordinary extension using a template.

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total cost.
        value_breakdown (dict): Cost breakdown for each service.
    """
    doc = Document(DOCX_PRORROGACAO_EXTRA_ORDINARIO_TEMPLATE_PATH)  # Load the single-class extraordinary extension template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
        if '{{ total_para_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_extra_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
                    if '{{ total_para_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_extra_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    docx_filename = f"Orçamento_{safe_trademark_name}_Prorrogacao_Extra_Ordinario_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file
    doc.save(docx_output_path)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

# Function to generate a DOCX file for a multi-class extraordinary extension
def generate_docx_from_prorrogacao_extra_ordinario_multi_class_template(marca_details, company_name, total_converted_sum, value_breakdown):
    """
    Generate a DOCX file for a multi-class extraordinary extension using a template.

    Parameters:
        marca_details (dict): Trademark details.
        company_name (str): Company name (Titular).
        total_converted_sum (float): Total cost.
        value_breakdown (dict): Cost breakdown for each service.
    """
    doc = Document(DOCX_PRORROGACAO_EXTRA_ORDINARIO_MULTI_CLASS_TEMPLATE_PATH)  # Load the multi-class extraordinary extension template

    # Helper function to safely retrieve values from the breakdown dictionary
    def get_value(service, key, default=0):
        return value_breakdown.get(service, {}).get(key, default)

    # Replace placeholders in paragraphs with actual data
    for paragraph in doc.paragraphs:
        if '{{ pais }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
        if '{{ data }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
        if '{{ titular }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
        if '{{ marca }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
        if '{{ taxa_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
        if '{{ taxa_prorrogacao_classe_adicional_extra_ordinario_usd }}' in paragraph.text:
            additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_classe_adicional_extra_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'extracted_val')))
        if '{{ total_prorrogacao_classe_adicional_extra_ordinario_usd }}' in paragraph.text:
            additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
            paragraph.text = paragraph.text.replace('{{ total_prorrogacao_classe_adicional_extra_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'total_converted')))
        if '{{ total_para_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
            paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_extra_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Replace placeholders in tables within the document
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    if '{{ pais }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ pais }}', marca_details['selected_country'])
                    if '{{ data }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ data }}', formatted_date)
                    if '{{ titular }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ titular }}', company_name)
                    if '{{ marca }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ marca }}', marca_details['trademark_name'])
                    if '{{ taxa_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        prorrogacao_label = next((label for label in marca_details['stored_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_extra_ordinario_usd }}', format_brazilian(get_value(prorrogacao_label, 'total_converted')))
                    if '{{ taxa_prorrogacao_classe_adicional_extra_ordinario_usd }}' in paragraph.text:
                        additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ taxa_prorrogacao_classe_adicional_extra_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'extracted_val')))
                    if '{{ total_prorrogacao_classe_adicional_extra_ordinario_usd }}' in paragraph.text:
                        additional_prorrogacao_label = next((label for label in marca_details['additional_class_labels'] if "Prorrogacao_Prazo_Extra_Ordinario" in label), None)
                        paragraph.text = paragraph.text.replace('{{ total_prorrogacao_classe_adicional_extra_ordinario_usd }}', format_brazilian(get_value(additional_prorrogacao_label, 'total_converted')))
                    if '{{ total_para_prorrogacao_extra_ordinario_usd }}' in paragraph.text:
                        paragraph.text = paragraph.text.replace('{{ total_para_prorrogacao_extra_ordinario_usd }}', format_brazilian(total_converted_sum))

    # Generate a safe filename by removing special characters and spaces
    safe_trademark_name = re.sub(r'[^\w\-]', '', marca_details['trademark_name'].replace(' ', ''))
    safe_country = re.sub(r'[^\w\-]', '', marca_details['selected_country'].replace(' ', ''))
    docx_filename = f"Orçamento_{safe_trademark_name}_Prorrogacao_Extra_Ordinario_{safe_country}_{filename_date}.docx"
    docx_output_path = os.path.join(output_dir, docx_filename)

    # Save the generated DOCX file
    doc.save(docx_output_path)
    print(f"DOCX file successfully generated and saved to: {docx_output_path}")

## 🧾 Trademark Budget Generator – Full Workflow Overview

This notebook implements a **complete, user-guided workflow** for generating trademark budget documents in both **PDF** (via LaTeX) and **DOCX** formats. Output files are generated dynamically based on user input and are designed for clarity, traceability, and operational readiness.

---

## 🎯 Purpose

- Automate the creation of trademark **budget summaries** and **procedural detail pages**.
- Generate professionally formatted **PDFs and DOCX files** for each trademark.
- Handle both **single** and **multi-trademark** budgeting scenarios.

---

## 🧑‍💻 User Experience Flow

- **Welcome Prompt:**  
  Collects the company name. If the input contains `"SAIR"`, the script exits gracefully.

- **Trademark Scope:**  
  Asks the user if the budget will cover **multiple trademarks**.

---

## ➕ Multi-Trademark Workflow

1. Determine whether **all trademarks** are for the **same country**:
   - If *yes*, collect the country and correspondent **once**.
2. For **each trademark**:
   - Invoke `collect_marca_details()`
   - Call `generate_marca_latex()` to build required LaTeX/DOCX structures.
   - Append generated content and metadata to in-memory lists for batch processing.

---

## ➖ Single-Trademark Workflow

- Collect input and generate all required files **once**, bypassing loop logic.

---

## 📝 DOCX Generation Logic

- Templates are chosen dynamically based on:
  - **Deposit type**: single-class vs multi-class
  - **Prorrogação** type: Ordinário vs Extraordinário
- The appropriate template rendering function is called based on decision branches.

---

## 📄 LaTeX PDF Compilation

- **Budget Summary:**
  - Concatenate LaTeX pages
  - Compile using `xelatex`
  - Save with standardized naming: `[trademark]_[country]_[date].pdf`

- **Procedural Details:**
  - Identical flow, based on `details_pages` list.

---

## ⚠️ Robustness & Logging

- File existence is validated prior to compilation to prevent overwrites.
- `xelatex` subprocess output is captured and logged.
- Errors in PDF generation are handled gracefully, with verbose logs for debugging.

---

## ✅ Final Outputs

- `📁` One **Budget PDF** per trademark
- `📁` One **Details PDF** per trademark
- `📁` One or more **DOCX** files
- `🧾` All output files are named using sanitized inputs for clarity and traceability


In [68]:
# Main script execution begins here
# Display welcome message and instructions
print("Bem-vindo à ferramenta de orçamento de Depósito e prorrogações.")
print("Para em qualquer momento finalizar seu Orçamento, digite 'SAIR'.")

# Collect the company name from the user
company_name = input("Digite o nome da empresa (Titular): ").strip()
if company_name.upper() == EXIT_KEYWORD:
    exit()  # Exit the script if the user enters the exit keyword
company_name = escape_latex(company_name)  # Escape special characters for LaTeX

# Ask if the user wants to budget multiple trademarks
while True:
    print("\nDeseja orçar múltiplas marcas?")
    options = ["Sim", "Não"]
    for i, opt in enumerate(options, 1):
        print(f"{i}. {opt}")
    multiple_marcas = input().strip().capitalize()
    if multiple_marcas.upper() == EXIT_KEYWORD:
        exit()
    if multiple_marcas.isdigit() and 1 <= int(multiple_marcas) <= len(options):
        multiple_marcas = options[int(multiple_marcas) - 1]
        break
    if multiple_marcas in options or multiple_marcas == "Nao":
        multiple_marcas = multiple_marcas
        break
    print("Entrada inválida. Digite o número (1 ou 2) ou a opção (Sim/Não).")

# Initialize lists to store generated content
latex_pages = []  # Store budget pages for PDF
details_pages = []  # Store details pages for PDF
docx_data_list = []  # Store data for DOCX generation

# Initialize variables to store the first trademark's details for PDF filenames
first_trademark_name = None
first_selected_country = None

if multiple_marcas == "Sim":
    # Ask if all trademarks are for the same country
    while True:
        print("\nTodas as marcas são para o mesmo país?")
        options = ["Sim", "Não"]
        for i, opt in enumerate(options, 1):
            print(f"{i}. {opt}")
        same_country = input().strip().capitalize()
        if same_country.upper() == EXIT_KEYWORD:
            exit()
        if same_country.isdigit() and 1 <= int(same_country) <= len(options):
            same_country = options[int(same_country) - 1]
            break
        if same_country in options or same_country == "Nao":
            same_country = same_country
            break
        print("Entrada inválida. Digite o número (1 ou 2) ou a opção (Sim/Não).")

    # Collect country and correspondent if all trademarks share the same country
    selected_country = None
    selected_correspondent = None
    if same_country == "Sim":
        # Step 1: Ask for country
        while True:
            countries = sorted(sheet_df['Pais_de_Atuacao'].dropna().unique())
            display_countries(countries, num_columns=5)
            country_input = input("\nQual o país onde será feito o depósito? ").strip()
            if country_input.upper() == EXIT_KEYWORD:
                exit()
            if country_input.isdigit() and 1 <= int(country_input) <= len(countries):
                selected_country = countries[int(country_input) - 1]
                break
            if country_input in countries:
                selected_country = country_input
                break
            print("Entrada inválida. Digite o número ou o nome do país.")

        # Step 2: Ask for correspondent
        while True:
            print("\nCorrespondentes disponíveis:")
            correspondents = sorted(sheet_df[sheet_df['Pais_de_Atuacao'] == selected_country]['Nome_do_Correspondente'].dropna().unique())
            for i, corr in enumerate(correspondents, 1):
                print(f"{i}. {corr}")
            corr_input = input("Qual o correspondente com o qual irá orçar? ").strip()
            if corr_input.upper() == EXIT_KEYWORD:
                exit()
            if corr_input.isdigit() and 1 <= int(corr_input) <= len(correspondents):
                selected_correspondent = correspondents[int(corr_input) - 1]
                break
            if corr_input in correspondents:
                selected_correspondent = corr_input
                break
            print("Entrada inválida. Digite o número ou o nome do correspondente.")

    # Collect details for each trademark
    while True:
        print("\nOrçamento para a próxima marca (digite 'SAIR' no nome da marca para finalizar):")
        marca_details = collect_marca_details(selected_country, selected_correspondent)
        if marca_details is None:
            break  # Exit loop if user chooses to stop
        # Store the first trademark's details for PDF filenames
        if not latex_pages:
            first_trademark_name = marca_details['trademark_name']
            first_selected_country = marca_details['selected_country']
        # Generate LaTeX content and DOCX data
        budget_page, details_page, docx_data = generate_marca_latex(marca_details, company_name)
        latex_pages.append(budget_page)
        details_pages.append(details_page)
        docx_data_list.append(docx_data)
else:
    # Collect details for a single trademark
    marca_details = collect_marca_details()
    if marca_details is not None:
        first_trademark_name = marca_details['trademark_name']
        first_selected_country = marca_details['selected_country']
        # Generate LaTeX content and DOCX data
        budget_page, details_page, docx_data = generate_marca_latex(marca_details, company_name)
        latex_pages.append(budget_page)
        details_pages.append(details_page)
        docx_data_list.append(docx_data)

# Generate appropriate DOCX files based on user input
if multiple_marcas == "Sim":
    for docx_data in docx_data_list:
        marca_details = docx_data['marca_details']
        if marca_details['deposit_option'] == "Depósito":
            if marca_details['num_classes'] == 1:
                generate_docx_from_template(
                    docx_data['marca_details'],
                    docx_data['company_name'],
                    docx_data['total_converted_sum'],
                    docx_data['value_breakdown'],
                    docx_data['stored_labels'],
                    docx_data['total_deposito'],
                    docx_data['total_concessao']
                )
            else:
                generate_docx_from_multi_class_template(
                    docx_data['marca_details'],
                    docx_data['company_name'],
                    docx_data['total_converted_sum'],
                    docx_data['value_breakdown'],
                    docx_data['stored_labels'],
                    docx_data['total_deposito'],
                    docx_data['total_concessao']
                )
        elif marca_details['deposit_option'] == "Prorrogação":
            # Determine if it's an ordinary or extraordinary extension
            is_ordinario = any("Prorrogacao_Prazo_Ordinario" in label for label in marca_details['stored_labels'])
            if marca_details['num_classes'] == 1:
                if is_ordinario:
                    generate_docx_from_prorrogacao_ordinario_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
                else:
                    generate_docx_from_prorrogacao_extra_ordinario_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
            else:
                if is_ordinario:
                    generate_docx_from_prorrogacao_ordinario_multi_class_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
                else:
                    generate_docx_from_prorrogacao_extra_ordinario_multi_class_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
else:
    # Generate DOCX for a single trademark
    docx_data = docx_data_list[0] if docx_data_list else None
    if docx_data:
        marca_details = docx_data['marca_details']
        if marca_details['deposit_option'] == "Depósito":
            if marca_details['num_classes'] == 1:
                generate_docx_from_template(
                    docx_data['marca_details'],
                    docx_data['company_name'],
                    docx_data['total_converted_sum'],
                    docx_data['value_breakdown'],
                    docx_data['stored_labels'],
                    docx_data['total_deposito'],
                    docx_data['total_concessao']
                )
            else:
                generate_docx_from_multi_class_template(
                    docx_data['marca_details'],
                    docx_data['company_name'],
                    docx_data['total_converted_sum'],
                    docx_data['value_breakdown'],
                    docx_data['stored_labels'],
                    docx_data['total_deposito'],
                    docx_data['total_concessao']
                )
        elif marca_details['deposit_option'] == "Prorrogação":
            # Determine if it's an ordinary or extraordinary extension
            is_ordinario = any("Prorrogacao_Prazo_Ordinario" in label for label in marca_details['stored_labels'])
            if marca_details['num_classes'] == 1:
                if is_ordinario:
                    generate_docx_from_prorrogacao_ordinario_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
                else:
                    generate_docx_from_prorrogacao_extra_ordinario_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
            else:
                if is_ordinario:
                    generate_docx_from_prorrogacao_ordinario_multi_class_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )
                else:
                    generate_docx_from_prorrogacao_extra_ordinario_multi_class_template(
                        docx_data['marca_details'],
                        docx_data['company_name'],
                        docx_data['total_converted_sum'],
                        docx_data['value_breakdown']
                    )

# Uncomment the following line to merge all generated DOCX files into one (currently disabled)
# merge_all_docx_files()

# Generate PDF documents from LaTeX content if any budgets were created
if latex_pages:
    # --- Generate Budget PDF ---
    # Remove \newpage from the last budget page to avoid an extra blank page
    latex_pages[-1] = latex_pages[-1].replace("\\newpage", "")
    latex_content = "".join(latex_pages)  # Combine all budget pages
    latex_output = latex_template_deposito % latex_content  # Insert into main template

    # Debug: Display the generated LaTeX content for verification
    print("Debug: Generated Budget LaTeX content:")
    print(latex_output)

    # Save the LaTeX content to a file for compilation
    with open('/content/sample.tex', 'w') as f:
        f.write(latex_output)

    # Copy the logo to the working directory for inclusion in the PDF
    if os.path.exists(LOGO_PATH):
        !cp "{LOGO_PATH}" "/content/logo.png"
    else:
        print(f"Error: PNG file not found at {LOGO_PATH}")
        exit()

    # Compile the LaTeX file to PDF using XeLaTeX, running twice to resolve references
    for _ in range(2):
        result = subprocess.run(['xelatex', '-interaction=nonstopmode', '/content/sample.tex'], capture_output=True)
    if result.returncode == 0 and os.path.exists('/content/sample.pdf'):
        # Generate a safe filename using the first trademark's details
        safe_trademark_name = re.sub(r'[^\w\-]', '', first_trademark_name.replace(' ', ''))
        safe_country = re.sub(r'[^\w\-]', '', first_selected_country.replace(' ', ''))
        pdf_filename = f"Orçamento_{safe_trademark_name}_{safe_country}_{filename_date}.pdf"
        output_path = os.path.join(output_dir, pdf_filename)
        !cp /content/sample.pdf "{output_path}"
        print(f"Budget PDF successfully generated and saved to: {output_path}")
    else:
        print("Error: Budget PDF generation failed. Check sample.log for details.")
        !cat /content/sample.log
        exit()

    # --- Generate Details PDF ---
    # Remove \newpage from the last details page to avoid an extra blank page
    details_pages[-1] = details_pages[-1].replace("\\newpage", "")
    details_content = "".join(details_pages)  # Combine all details pages
    details_output = latex_template_details % details_content  # Insert into main template

    # Debug: Display the generated LaTeX content for verification
    print("Debug: Generated Details LaTeX content:")
    print(details_output)

    # Save the LaTeX content to a file for compilation
    with open('/content/details.tex', 'w') as f:
        f.write(details_output)

    # Ensure the logo is available in the working directory
    if not os.path.exists('/content/logo.png'):
        !cp "{LOGO_PATH}" "/content/logo.png"

    # Compile the LaTeX file to PDF using XeLaTeX, running twice to resolve references
    for _ in range(2):
        result = subprocess.run(['xelatex', '-interaction=nonstopmode', '/content/details.tex'], capture_output=True)
    if result.returncode == 0 and os.path.exists('/content/details.pdf'):
        # Generate a filename for the details PDF
        details_pdf_filename = f"Detalhes_{pdf_filename}"
        details_output_path = os.path.join(output_dir, details_pdf_filename)
        !cp /content/details.pdf "{details_output_path}"
        print(f"Details PDF successfully generated and saved to: {details_output_path}")
    else:
        print("Error: Details PDF generation failed. Check details.log for details.")
        !cat /content/details.log
        exit()
else:
    # Inform the user if no budgets were generated
    print("\nNenhum orçamento gerado.")

Bem-vindo à ferramenta de orçamento de Depósito e prorrogações.
Para em qualquer momento finalizar seu Orçamento, digite 'SAIR'.
Digite o nome da empresa (Titular): Clients Company Name

Deseja orçar múltiplas marcas?
1. Sim
2. Não
1

Todas as marcas são para o mesmo país?
1. Sim
2. Não
2

Orçamento para a próxima marca (digite 'SAIR' no nome da marca para finalizar):

Países disponíveis:
                               Grupo 1                    Grupo 2        Grupo 3                   Grupo 4                         Grupo 5
                        1. AFEGANISTAO                 27. CHIPRE     53. GRECIA              79. MARROCOS                     105. RUSSIA
                      2. AFRICA DO SUL               28. COLOMBIA  54. GUATEMALA                80. MEXICO 106. SAO TOME E PRINCIPE AFRICA
                            3. ALBANIA    29. COMUNIDADE EUROPEIA     55. GUIANA            81. MOCAMBIQUE                 107. SERRA LEOA
                           4. ALEMANHA        30. CO

## 📬 Contact

For questions, suggestions, or collaboration inquiries, please feel free to reach out:

- **👤 Author**: Martin G. Lartigue  
- **📧 Email**: [martin.g.lartigue@gmail.com](mailto:martin.g.lartigue@gmail.com)

I'm open to feedback, feature requests, and contributions related to this project or other automation initiatives in the IP (Intellectual Property) and document generation space.

# Thank You

I sincerely appreciate all the support and eventual feedback.